# Chess games analyze
**Задача: разработать модель для предсказания следующего хода для любой возможной позиции шахматной партии.**

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import os
import pandas as pd
import numpy as np
import chess
import torch
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import train_test_split
import pipreqs
import torch.nn.functional as F

from src.data.preprocessor import fen_to_tensor, move_to_target
from src.data.masks import get_position_masks
from src.data.dataset import ChessDataset
from src.model.model import ChessModel
from src.training.trainer import train_model
from src.test.test import test_model

In [31]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
path = r"data\fens_training_set.csv"

In [33]:
if os.path.exists(path):
    data = pd.read_csv(path)
else:
    print('Something is wrong')
    data = None

### Входные данные
- **Позиции** в формате FEN (Forsyth–Edwards Notation)
- **Ходы** в формате UCI (Universal Chess Interface)

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268550 entries, 0 to 268549
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   fen     268550 non-null  object
 1   move    268550 non-null  object
dtypes: object(2)
memory usage: 4.1+ MB


In [35]:
data.head()

,fen,move
0,7k/5p1p/p2p1Pr1/1p4pQ/8/P1P5/2pr3P/2R2K2 w - -...,h5f3
1,5r2/2p3k1/1p3pb1/p1pPr2p/2P1PRP1/3B1R2/PP4K1/8...,g4h5
2,r2r2k1/p3bpp1/q2Bp2p/2pn4/2b1N1Q1/8/PPP2PPP/3R...,d6e7
3,8/8/8/7K/1k6/8/6P1/8 w - - 0 1,g2g4
4,rnbqkb1r/pp3ppp/5n2/3p4/2pP4/3B1N2/PPP2PPP/RNB...,f1e1


In [36]:
data = data.sample(n=1000)
data.shape

(1000, 2)

### Решение

- Предлагается представить расстановку фигур на доске как тензор (8х8х18), где 18 - число "плоскостей" для фигур каждого типа и цвета, а так же возможные ситуации: рокировки, взятие на проходе, превращения пешки.

- Таргет (ход) будет представлен тремя векторами: from_target (64) - откуда ходить, to_target (64) - куда ходить, promotion_target (5) - возможные ситуации.

- Соответственно будет решаться задача многоклассовой классификации. 
Для этого используем нейросеть CNN архитектуры с многоголовым выходом.

#### Особенности:


- Рализована функция get_position_masks(), которая возвращает маски легальных ходов.


- Реальзована функция apply_masks(), которая применяет маски к предсказаниям модели при обучении. Это позволяет модели "не отвлекаться" на заведомо неверные позиции. Это ускорит сходимость обучения. Предотвращает "зашумление" нелегальными ходами, улучшает обобщающую способность модели.

- Для инференса гарантируются легальные предсказания, повышается надежность.

### Метрики

- Cross Entropy Loss при обучении. Максимизирует вероятность правильного ответа.

- Для данной задачи главными метриками качества будут accuracy, Top-K.

- Accuracy: доля случаев, когда наиболее вероятный ход, предсказанный моделью, совпадает с реальным ходом (Top‑1 accuracy). Показывает, как часто модель угадывает ход без учета других вариантов.

- Top‑K: доля случаев, когда истинный ход входит в K наиболее вероятных предсказаний модели.
(Как првило используется Top-3. Top-5 или Top-10 могут быть завышены за счет распространенных ходов)

- Log Loss / Perplexity (в данной работе не рассматривались): оценка качества вероятностей.

In [37]:
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

In [38]:
train_dataset = ChessDataset(train_df)
test_dataset = ChessDataset(test_df)

In [39]:
train_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=4
)

test_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=4
)

In [40]:
train_model(train_loader, DEVICE)

Batch 0, Loss: 2.717
Модель сохранена: models/best_model.pth
Новая лучшая модель. Loss: 3.4712
Epoch 0, Train Loss: 3.4712
Accuracy: from 0.0088, to 0.0262, prom 0.0262
Top-K: from 0.0200, to 0.0725, prom 0.4075
Batch 0, Loss: 3.038
Модель сохранена: models/best_model.pth
Новая лучшая модель. Loss: 3.2061
Epoch 1, Train Loss: 3.2061
Accuracy: from 0.0050, to 0.0325, prom 0.1138
Top-K: from 0.0275, to 0.0812, prom 0.3500
Batch 0, Loss: 3.367
Модель сохранена: models/best_model.pth
Новая лучшая модель. Loss: 3.1669
Epoch 2, Train Loss: 3.1669
Accuracy: from 0.0075, to 0.0400, prom 0.0075
Top-K: from 0.0225, to 0.1037, prom 0.3100
Batch 0, Loss: 2.936
Модель сохранена: models/best_model.pth
Новая лучшая модель. Loss: 3.1478
Epoch 3, Train Loss: 3.1478
Accuracy: from 0.0088, to 0.0362, prom 0.0075
Top-K: from 0.0237, to 0.0913, prom 0.1513
Batch 0, Loss: 2.873
Модель сохранена: models/best_model.pth
Новая лучшая модель. Loss: 3.1395
Epoch 4, Train Loss: 3.1395
Accuracy: from 0.0050, to 0.0

In [41]:
test_model(test_loader, DEVICE)

Модель загружена: models/best_model.pth
Средний Loss: 3.3673
Accuracy from 0.005
Accuracy to 0.005
Accuracy prom 0.000
Top-3 from 0.015
Top-3 to 0.035
Top-3 prom 0.175


{'loss': 3.367342052459717,
 'accuracy_from': tensor(0.0050),
 'accuracy_to': tensor(0.0050),
 'accuracy_prom': tensor(0.),
 'top3_from': tensor(0.0150),
 'top3_to': tensor(0.0350),
 'top3_prom': tensor(0.1750)}

### Стиль игрока

Для решения такой задачи требуется учесть особенности стиля игры. Спрогнозировать не лучший ход, а какой ход сделает игрок.

#### Особенности стиля игрока

- Стиль может заключаться в ряде паттернов: предпочтения в дебютах, склонность к атаке или защите, рискованность решений, частота жертв, темп игры, типичные ошибки и частота неочевидных ходов. Эти признаки можно извлечь из истории его партий (например, из PGN/FEN данных).

#### Моделирование

- Оптимальная стратегия — двухэтапное обучение:

- Сначала обучаем общую модель на большой выборке (100К - 1М).

- Fine-tuning на партиях конкретного игрока. При достаточном объёме данных (≈1000+ партий) это позволяет адаптировать веса модели под индивидуальный стиль.

- Альтернативный вариант — ввести вектор игрока (player embedding) как дополнительный вход в модель. Тогда модель учится различать стили разных игроков даже при ограниченных данных.

#### Отличия индивидуальной модели от общей

Отличия: общая модель ищет объективно лучший ход. Индивидуальная предсказывает ход конкретного игрока.

#### Метрики оценки

Для оценки применяются те же метрики, что и для общей модели: Cross Entropy Loss при обучении, Top-k Accuracy и Log Loss / Perplexity на валидации. Дополнительно можно сравнивать точность персонализированной модели с общей — насколько часто она точнее угадывает ходы данного игрока.